In [2]:
import huggingface_hub
import os
import sys
import sentence_transformers


# download intfloat/multilingual-e5-base



/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import torch
from mteb import MTEB
from tasks import *
from transformers import AutoTokenizer, AutoModel

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print("Using device:", device)

# load from disk
class embedding_model:
    def __init__(self, model, tokenizer, device, inference=False):
        self.model = model.to(device)
        self.tokenizer = tokenizer
        self.device = device
        if inference:
            self.model.eval()
            self.model.requires_grad_(False)

    def average_pool(last_hidden_states: torch.Tensor,
                 attention_mask: torch.Tensor) -> torch.Tensor:
        last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
        return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]
    
    def encode(self, sentences, batch_size=512):
        embeddings = []
        for i in range(0, len(sentences), batch_size):
            batch_sentences = sentences[i:i+batch_size]
            embedding = self.__call__(batch_sentences)
            # Add each sentence embedding to the list as a separate ndarray
            embeddings.extend(embedding.detach().cpu().numpy())
        return embeddings

    def __call__(self, data):
        tokens_and_mask = self.tokenizer(data, return_tensors='pt', padding=True, truncation=True, max_length=512).to(self.device)
        model_output = self.model(tokens_and_mask["input_ids"], attention_mask=tokens_and_mask["attention_mask"])
        embedding = embedding_model.average_pool(model_output.last_hidden_state, attention_mask=tokens_and_mask["attention_mask"])
        # normalize the embedding
        embedding = embedding / embedding.norm(dim=-1, keepdim=True)
        return embedding    

a = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-base")
# save a to models/raw_models/multilingual-e5-base
save_path = "models/tokenizers/multilingual-e5-base"
a.save_pretrained(save_path)


Using device: cuda


tokenizer_config.json: 100%|██████████| 418/418 [00:00<00:00, 1.04MB/s]
sentencepiece.bpe.model: 100%|██████████| 5.07M/5.07M [00:00<00:00, 176MB/s]
tokenizer.json: 100%|██████████| 17.1M/17.1M [00:00<00:00, 374MB/s]
special_tokens_map.json: 100%|██████████| 280/280 [00:00<00:00, 2.66MB/s]


('models/tokenizers/multilingual-e5-base/tokenizer_config.json',
 'models/tokenizers/multilingual-e5-base/special_tokens_map.json',
 'models/tokenizers/multilingual-e5-base/sentencepiece.bpe.model',
 'models/tokenizers/multilingual-e5-base/added_tokens.json',
 'models/tokenizers/multilingual-e5-base/tokenizer.json')